In [13]:
import urllib.request
import nrel
import json

In [8]:
r = requests.get("https://developer.nrel.gov/api/alt-fuel-stations/v1.json?fuel_type=E85,ELEC&state=CA&api_key=" + nrel.nrel_api['api_key'] + "&format=JSON")

In [16]:
with open("nrel_data.json", "w") as write_file:
    with urllib.request.urlopen("https://developer.nrel.gov/api/alt-fuel-stations/v1.json?fuel_type=E85,ELEC&state=CA&api_key=" + nrel.nrel_api['api_key'] + "&format=JSON") as url:
        data = json.loads(url.read().decode())
        json.dump(data, write_file)

In [17]:
write_file.close()